In [60]:
import cirq
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq
import pandas as pd

from qite import QITE
from qbm import QBM
from circuit import build_ansatz, initialize_ansatz_symbols
from problem import build_ising_model_hamiltonian
from hamiltonian import Hamiltonian
from utils import evaluate_exact_state, plot_density_matrix_heatmap, get_ancillary_qubits, save_circuit_to_svg, circuit_to_state
from dataset import bars_and_stripes_probability, bars_and_stripes, samples_from_distribution, plot_dataset

## Define dataset

Define dataset, that is the bar and stripes on 2x2 grid and for simplifying we consider empty and fully filled as invalid values 

In [73]:
p_data = tf.convert_to_tensor(
    bars_and_stripes_probability(bars_and_stripes(n=100000, no_fills_or_empties=True))
)
print(np.around(p_data.numpy(),2))

[0.   0.   0.   0.25 0.   0.25 0.   0.   0.   0.   0.25 0.   0.25 0.
 0.   0.  ]


## Define Hamiltonian and qubits

Define Hamiltonian of which coefficients are to be trained so that the thermal state's sampling probabilties would be as close as possible to the data distribution

In [6]:
ising_model, problem_qubits = build_ising_model_hamiltonian(
    [4], transverse=None
)
ancillary_qubits = get_ancillary_qubits(problem_qubits)
qubits = [*problem_qubits, *ancillary_qubits]
initial_coefficients = tf.random.uniform(
    [len(ising_model)], minval=-1, maxval=1
)
hamiltonian = Hamiltonian(
    ising_model,
    coefficients=tf.Variable(
        tf.identity(initial_coefficients)
    ),
)

## Define the quantum circuit

Define the quantum circuit that is used for VarQITE and the preparation of the thermal state

In [71]:
n_layers = 3
circuit, symbol_names = build_ansatz(qubits, n_layers=n_layers)
initial_symbol_values = initialize_ansatz_symbols(
    len(qubits), 
    n_layers=n_layers
)

## Setup VarQBM model

Setup a VarQBM model using mostly defaults, just created circuit, Hamiltonian, circuit symbols and their initial value, and the number of time steps to which the VarQITE is split to inside VarQBM (higher value generally is more accurate but takes more time)

In [ ]:
qbm = QBM(
    circuit,
    symbol_names,
    initial_symbol_values,
    hamiltonian,
    n_timesteps=40,
    verbose_qite=3,
)

## Run VarQBM training
Running VarQBM training gives us the Hamiltonian with trained coefficients, the final trained state in density matrix format, trained circuit symbol values and metrics from the training process. (Likely requires redesign for actual hardware, i.e. for the density matrix one could directly here return samples if possible.)

In [7]:
# trained_hamiltonian, trained_state, trained_symbol_values, metrics = qbm.train(p_data, epochs=40)

(To avoid running several hours, here's the result of one "succesful" run)

In [72]:
trained_hamiltonian_coefficients = [
    -0.13697385787963867,
    -0.15982627868652344,
    1.1702172756195068,
    2.6191375255584717,
    -0.29886317253112793,
    -0.29584717750549316,
    -0.07853007316589355,
    -0.2576768398284912,
    -0.2590906620025635,
    -0.014436483383178711
]
trained_hamiltonian = Hamiltonian(ising_model, trained_hamiltonian_coefficients)
# One can prepare the thermal state of the trained Hamiltonian again by running VarQITE
#trained_symbol_values, state = qbm.run_qite(trained_hamiltonian, skip_metrics=True)
trained_symbol_values = [
    -0.014642548747360706,
    -3.441415117322322e-07,
    -0.01728099398314953,
    -8.662656000524294e-06,
    -0.004501198884099722,
    1.1648027793853544e-05,
    0.2876802682876587,
    -0.00012912784586660564,
    -1.3544297627898771e-11,
    -6.415218933852884e-08,
    -1.2945443328415962e-11,
    -3.1357657803710026e-08,
    -1.0079138912377772e-11,
    -7.583848571357521e-08,
    -1.781818442792016e-11,
    -2.060415482674216e-07,
    -0.014686104841530323,
    -9.479756045038812e-06,
    -0.016323236748576164,
    1.5828969480935484e-05,
    0.041991300880908966,
    -0.00016045317170210183,
    0.07910387963056564,
    -0.0002838200598489493,
    -3.075627599824493e-11,
    -3.1359231655869735e-08,
    -1.2116604594658575e-11,
    -7.584274896998977e-08,
    -1.7822255823918276e-11,
    -2.5283253535235417e-07,
    -1.7819704045685114e-11,
    -2.0604063877271983e-07,
    -0.016164422035217285,
    2.3528562451247126e-05,
    0.009831184521317482,
    -0.0001395646104356274,
    -1.051080584526062,
    -0.00016350357327610254,
    0.015614356845617294,
    -0.00030664922087453306,
    2.5852771312617762e-11,
    -1.4341814846829948e-07,
    1.8851156746713116e-11,
    -1.1220399187550356e-07,
    1.2182884909228697e-11,
    1.3185865554987686e-07,
    1.2183080932981483e-11,
    1.3185872660415043e-07,
    1.578497290611267,
    -8.13862470749882e-07,
    1.565839171409607,
    -7.825872307876125e-05,
    1.509065866470337,
    -0.00017533988284412771,
    1.5611470937728882,
    -4.3912779801758006e-05,
    -5.514004183804211e-12,
    3.741260456990858e-08,
    3.3556379896992894e-11,
    1.1650548259467541e-07,
    4.171089515447868e-11,
    -5.5527948461531196e-08,
    4.2645778401684264e-12,
    -5.552934823072064e-08,
    0.999977171421051,
    0.9999960064888,
    0.999805212020874,
    1.0,
    1.0,
    1.0,
    1.0,
    0.9999657273292542,
    0.9997722506523132,
    0.999398410320282,
    1.0,
    1.0,
    1.0,
    1.0,
    0.9995574355125427,
    0.9996995329856873,
    0.999622642993927,
    1.0,
    1.0,
    1.0,
    1.0
]

## Pulling samples

Cirq and Tfq provides many different ways to get samples given the circuit, symbols and their values. One examples shown below which is easy but relies potentially unrealistic features.

### Tfq state layer to get probability distribution
Use ``circuit_to_state`` utility function which internally uses Tfq's State layer to get density matrix from which the probability distribution can be easilly acquired

In [74]:
state = circuit_to_state(
    circuit,
    symbol_names,
    trained_symbol_values,
    list(range(len(problem_qubits))),
)
p_model = np.diag(state.numpy().real)

Draw samples from probability distribution

In [64]:
plot_dataset(samples_from_distribution(p_model), file_format="png", size = 2)

Another way (potentially more "realistic") is to use Cirq's simulator to get samples from the problem qubits (i.e. tracing out the ancillary qubits). 